In [ ]:
from Loopy_sim import LoopySim
from Lunacy_class import LunacyMultiTool, LunacyMulticlassifier
import matplotlib.pyplot as plt

import numpy as np

import torch.nn as nn
import torch

set number of lobes for Loopy's morphology, and then create the Loopy simulator

In [ ]:
num_lobes = 4

sim = LoopySim(lobes=num_lobes)
datafile = f'data/loopy_{num_lobes}L_dataset.csv'


generate the loopy dataset for the given morphology

In [ ]:
sim.loopy_dataset_generator(dx=5, dy=5, dtheta=10, filename=datafile)

visualize the generated dataset of the given morphology

In [ ]:
print('noise')
sim.visualize_label_from_csv(datafile, 'noise', num_samples=1500, plot_height=100, save_image=False)
print('predator')
sim.visualize_label_from_csv(datafile, 'predator', num_samples=1500, plot_height=100, save_image=False)
print('prey')
sim.visualize_label_from_csv(datafile, 'prey', num_samples=1500, plot_height=100, save_image=False)

background_image = sim.predator_image_creator()
x_coords, y_coords = sim.generate_loopy_coords()
sim.overlay_loopy_on_image(background_image, x_coords, y_coords, x_pos=50, y_pos=50, rotation=0, plot=True)

define the network architecture and load in the data

In [ ]:
input_size = 36  
single_layers = 17
num_multi_layers = 18-single_layers  
num_classes = 3

model = LunacyMulticlassifier(input_size, num_classes, single_layers, num_multi_layers)
lunacy = LunacyMultiTool(datafile, model, sim)

Train the network

In [ ]:
lunacy.train_model(num_epochs=10000, lr=0.01, directory="multi_weights", class_labels=None)

Load in the best weights, that I have trained and visualize a validation data sample

In [ ]:
lunacy.load_weights(f'best_weights\lunacy_multi_single_4L_80_83_86.pth')

fig, ax = plt.subplots()
lunacy.visualize_with_loopy(150, ax)

perform the cellular level microscopic validations for the model

In [ ]:
results = lunacy.micro_validate_model()

print("Confusion Matrix:\n", results["confusion_matrix"])
print("Accuracy:", results["accuracy"])
print("Precision:", results["precision"])
print("Recall:", results["recall"])
print("F1 Score:", results["f1_score"])


perform the loopy level macroscopic validations for the model with a given threshold for minimum concensus

In [ ]:
results = lunacy.macro_validate_model(threshold=50, plot=True)

print("Confusion Matrix:\n", results["confusion_matrix"])
print("Accuracy:", results["accuracy"])
print("Precision:", results["precision"])
print("Recall:", results["recall"])
print("F1 Score:", results["f1_score"])

plot the validation metrics over the minimum consensus threshold for the given loopy morphology

In [ ]:
thresholds = np.arange(0, 101, 10)
accuracies = []
precisions = []
recalls = []
f1_scores = []

for threshold in thresholds:
    results = lunacy.macro_validate_model(threshold=threshold, plot=False)
    accuracies.append(results['accuracy'])
    precisions.append(results['precision'])
    recalls.append(results['recall'])
    f1_scores.append(results['f1_score'])

# Plotting
plt.figure(figsize=(10, 6))
plt.plot(thresholds, accuracies, label='Accuracy', marker='o')
plt.plot(thresholds, precisions, label='Precision', marker='s')
plt.plot(thresholds, recalls, label='Recall', marker='^')
plt.plot(thresholds, f1_scores, label='F1 Score', marker='d')

plt.xlabel('Threshold (%)')
plt.ylabel('Performance Metric Value')
plt.title('Macro Metrics vs. Threshold')
plt.legend()
plt.grid(True)
plt.xticks(thresholds)
plt.ylim(0, 1)
plt.show()

Load in all the best weights across all three morphologies

In [ ]:
input_size = 36  
single_layers = 17
num_multi_layers = 18-single_layers  
num_classes = 3

num_lobes = 5
datafile = f'data/loopy_{num_lobes}L_dataset.csv'
sim = LoopySim(lobes=num_lobes)
model = LunacyMulticlassifier(input_size, num_classes, single_layers, num_multi_layers)
lunacy5 = LunacyMultiTool(datafile, model, sim)
lunacy5.load_weights(f'best_weights\lunacy_multi_single_5L_83_85_83_80.pth')

num_lobes = 4
datafile = f'data/loopy_{num_lobes}L_dataset.csv'
sim = LoopySim(lobes=num_lobes)
model = LunacyMulticlassifier(input_size, num_classes, single_layers, num_multi_layers)
lunacy4 = LunacyMultiTool(datafile, model, sim)
lunacy4.load_weights(f'best_weights\lunacy_multi_single_4L_80_83_86.pth')

num_lobes = 3
datafile = f'data/loopy_{num_lobes}L_dataset.csv'
sim = LoopySim(lobes=num_lobes)
model = LunacyMulticlassifier(input_size, num_classes, single_layers, num_multi_layers)
lunacy3 = LunacyMultiTool(datafile, model, sim)
lunacy3.load_weights(f'best_weights\lunacy_multi_single_3L_81_74_83_87.pth')

perform all validations across all morphologies with all the best weights

In [ ]:
# Perform micro validation for each morphology
micro_metrics_5 = lunacy5.micro_validate_model(plot=False)
micro_metrics_4 = lunacy4.micro_validate_model(plot=False)
micro_metrics_3 = lunacy3.micro_validate_model(plot=False)

# Extract metrics
morphologies = ['5 Lobes', '4 Lobes', '3 Lobes']
micro_accuracies = [
    micro_metrics_5["accuracy"],
    micro_metrics_4["accuracy"],
    micro_metrics_3["accuracy"]
]
micro_precisions = [
    micro_metrics_5["precision"],
    micro_metrics_4["precision"],
    micro_metrics_3["precision"]
]
micro_recalls = [
    micro_metrics_5["recall"],
    micro_metrics_4["recall"],
    micro_metrics_3["recall"]
]
micro_f1_scores = [
    micro_metrics_5["f1_score"],
    micro_metrics_4["f1_score"],
    micro_metrics_3["f1_score"]
]
micro_specificities = [
    micro_metrics_5["specificity"],
    micro_metrics_4["specificity"],
    micro_metrics_3["specificity"]
]


# Updated font size
font_size = 20

# Apply font size to all relevant plot elements
plt.rc('font', size=font_size)          # controls default text sizes
plt.rc('axes', titlesize=font_size)     # fontsize of the axes title
plt.rc('axes', labelsize=font_size)     # fontsize of the x and y labels
plt.rc('xtick', labelsize=font_size)    # fontsize of the tick labels
plt.rc('ytick', labelsize=font_size)    # fontsize of the tick labels
plt.rc('legend', fontsize=font_size)    # legend fontsize
plt.rc('figure', titlesize=font_size)   # fontsize of the figure title


# Plot micro metrics bar chart
x = np.arange(len(morphologies))
width = 0.15

plt.figure(figsize=(12, 6))
plt.bar(x - 2 * width, micro_accuracies, width, label='Accuracy')
plt.bar(x - width, micro_precisions, width, label='Precision')
plt.bar(x, micro_recalls, width, label='Recall')
plt.bar(x + width, micro_f1_scores, width, label='F1 Score')
plt.bar(x + 2 * width, micro_specificities, width, label='Specificity')

plt.xlabel('Morphologies')
plt.ylabel('Metric Value')
plt.title('Micro Validation Metrics Across Morphologies')
plt.xticks(x, morphologies)
plt.ylim(0, 1)
plt.legend(loc='lower left')
plt.grid(axis="y")
plt.show()

# Perform macro validation for each morphology across thresholds
thresholds = np.arange(10, 101, 10)
macro_metrics_5 = [lunacy5.macro_validate_model(threshold, plot=False) for threshold in thresholds]
macro_metrics_4 = [lunacy4.macro_validate_model(threshold, plot=False) for threshold in thresholds]
macro_metrics_3 = [lunacy3.macro_validate_model(threshold, plot=False) for threshold in thresholds]

# Extract macro metrics
macro_accuracies = {
    "5 Lobes": [metrics["accuracy"] for metrics in macro_metrics_5],
    "4 Lobes": [metrics["accuracy"] for metrics in macro_metrics_4],
    "3 Lobes": [metrics["accuracy"] for metrics in macro_metrics_3]
}
macro_precisions = {
    "5 Lobes": [metrics["precision"] for metrics in macro_metrics_5],
    "4 Lobes": [metrics["precision"] for metrics in macro_metrics_4],
    "3 Lobes": [metrics["precision"] for metrics in macro_metrics_3]
}
macro_recalls = {
    "5 Lobes": [metrics["recall"] for metrics in macro_metrics_5],
    "4 Lobes": [metrics["recall"] for metrics in macro_metrics_4],
    "3 Lobes": [metrics["recall"] for metrics in macro_metrics_3]
}
macro_f1_scores = {
    "5 Lobes": [metrics["f1_score"] for metrics in macro_metrics_5],
    "4 Lobes": [metrics["f1_score"] for metrics in macro_metrics_4],
    "3 Lobes": [metrics["f1_score"] for metrics in macro_metrics_3]
}
macro_specificities = {
    "5 Lobes": [metrics["specificity"] for metrics in macro_metrics_5],
    "4 Lobes": [metrics["specificity"] for metrics in macro_metrics_4],
    "3 Lobes": [metrics["specificity"] for metrics in macro_metrics_3]
}





# Color and shape code for morphologies
colors = {'3 Lobes': 'red', '4 Lobes': 'blue', '5 Lobes': 'green'}
markers = {'Accuracy': 'o', 'Precision': 's', 'Recall': '^', 'F1 Score': 'd', 'Specificity': 'x'}

# Plot macro metrics across thresholds for each morphology
plt.figure(figsize=(14, 8))
for morphology, color in colors.items():
    for metric_name, metric_dict in zip(
        ['Accuracy', 'Precision', 'Recall', 'F1 Score', 'Specificity'],
        [macro_accuracies, macro_precisions, macro_recalls, macro_f1_scores, macro_specificities]
    ):
        plt.plot(thresholds, metric_dict[morphology], label=f'{morphology} - {metric_name}',
                 color=color, marker=markers[metric_name])

plt.xlabel('Threshold (%)')
plt.ylabel('Metric Value')
plt.title('Macro Validation Metrics Across Thresholds and Morphologies')
plt.xticks(thresholds)
plt.yticks(np.arange(0, 1.1, 0.1))
plt.ylim(0, 1)
plt.legend(loc='lower left', ncol=3,fontsize=18)
plt.grid(axis="y")
plt.tight_layout()
plt.show()

create a video of validation sets with loopy's preditions and locations

In [ ]:
sample_indices = range(0, 1000)

lunacy3.animate_samples(sample_indices, interval=500, save_as='lunacy3_validation.mp4')
lunacy4.animate_samples(sample_indices, interval=500, save_as='lunacy4_validation.mp4')
lunacy5.animate_samples(sample_indices, interval=500, save_as='lunacy5_validation.mp4')